<a href="https://colab.research.google.com/github/7ssasbiat/Bitcoin-Price-Prediction-Using-Multivariate-Time-Series-/blob/main/Bitcoin_Price_Prediction_Using_Multivariate_Time_Series_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BITCOIN PRICE FORCASTING USING VAR,VARMA AND VARMA with AUTO-ARIMA


# **INTRODUCTION**

In this project, we are going to choose the best model beteween the following multivariate time series models : VAR,VARMA AND VARMA with AUTO-ARIMA to forcast bitcoin price.

Why MTS?

 Multivariate Time Series consist of more than one time-dependent variable and each variable depends not only on its past values but also has some dependency on other variables.

Bitcoin’s movements show now a high correlation with those of Nasdaq index and S&P500. That's why we will validate if there is a real causality relation between them so we can use multivariate time series models for forecasting.


  



# **Import Necessary Libraries**

We should get the data we will work on, so let's instal **yfinance** library to get bitcoin dataset from 2017-03-21 to 2022-03-21:

In [ ]:
pip install yfinance

     |████████████████████████████████| 63 kB 873 kB/s 
     |████████████████████████████████| 6.4 MB 18.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install statsmodels==0.11 #this version is necessary to work with pmdarima library

     |████████████████████████████████| 8.7 MB 6.9 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
pip install pmdarima

     |████████████████████████████████| 1.4 MB 8.8 MB/s 


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
import statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

import sklearn
from sklearn.model_selection import ParameterGrid
from sklearn import metrics
from sklearn.model_selection import ParameterGrid

from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX
from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore")
from pmdarima.arima import auto_arima


# **Dataset Construction**

### **A. BTC Dataset** 

In [ ]:
data= yf.download("BTC-USD", start="2017-03-21", end="2022-03-21")
data=data.rename(columns={"High": "BTC High",
                   "Volume": "BTC Volume"})
data.drop(['Open','Low','Close','Adj Close','BTC Volume'], axis='columns', inplace=True) #We will work with high value only
data = data.reset_index(level=0) # to make the date a variable
data['Date'] = data['Date'].dt.strftime('%m/%d/%Y') # to make the date format like in the next dataset
data

[*********************100%***********************]  1 of 1 completed


,Date,BTC High
0,03/21/2017,1122.430054
1,03/22/2017,1120.650024
2,03/23/2017,1058.010010
3,03/24/2017,1040.469971
4,03/25/2017,975.760986
...,...,...
1822,03/17/2022,41287.535156
1823,03/18/2022,42195.746094
1824,03/19/2022,42316.554688
1825,03/20/2022,42241.164062


### **B. S&P500 and NASDAQ Dataset** 

To get the SP500&NASDAQ datasets, we used the one given by Yahoo Finance.
You can download it from [SP500 dataset](https://finance.yahoo.com/quote/%5EGSPC/history/) and [NASDAQ dataset](https://finance.yahoo.com/quote/%5EIXIC/history?period1=1490054400&period2=1647820800&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true).

In [ ]:
from google.colab import files 
import io 
uploaded = files.upload()

Saving ^IXIC.csv to ^IXIC.csv
Saving SP500.csv to SP500.csv


In [ ]:
df=pd.read_csv('SP500.csv')
dfn=pd.read_csv('^IXIC.csv')
dfn['Date'] = pd.to_datetime(dfn['Date'])
dfn['Date'] = dfn['Date'].dt.strftime('%m/%d/%Y') # to make the date format like the others
dfn

,Date,Open,High,Low,Close,Adj Close,Volume
0,03/21/2017,5923.419922,5928.060059,5790.729980,5793.830078,5793.830078,2192090000
1,03/22/2017,5790.589844,5825.669922,5781.799805,5821.640137,5821.640137,1857680000
2,03/23/2017,5812.310059,5842.819824,5806.979980,5817.689941,5817.689941,1742640000
3,03/24/2017,5839.330078,5858.950195,5807.830078,5828.740234,5828.740234,1841920000
4,03/27/2017,5776.330078,5849.200195,5769.390137,5840.370117,5840.370117,1670760000
...,...,...,...,...,...,...,...
1254,03/14/2022,12795.120117,12918.009766,12555.349609,12581.219727,12581.219727,5853360000
1255,03/15/2022,12685.230469,12973.879883,12616.589844,12948.620117,12948.620117,5414590000
1256,03/16/2022,13119.370117,13440.120117,12992.200195,13436.549805,13436.549805,6498110000
1257,03/17/2022,13360.719727,13620.799805,13317.139648,13614.780273,13614.780273,5575030000


 >SP500 and NASDAQ dataset cleaning

In [ ]:
df=df.rename(columns={"High": "SP500 High"})
df.drop(['Open','Low','Close','Unnamed: 0'], axis='columns', inplace=True) # Since we are working only with high value
df = df.iloc[::-1] # to sort the SP500 dataset the same way as BTC dataset 
dfn=dfn.rename(columns={"High": "NASDAQ High"})
dfn.drop(['Open','Low','Close','Adj Close','Volume'], axis='columns', inplace=True) # Since we are working only with high value
dfn

,Date,NASDAQ High
0,03/21/2017,5928.060059
1,03/22/2017,5825.669922
2,03/23/2017,5842.819824
3,03/24/2017,5858.950195
4,03/27/2017,5849.200195
...,...,...
1254,03/14/2022,12918.009766
1255,03/15/2022,12973.879883
1256,03/16/2022,13440.120117
1257,03/17/2022,13620.799805


### **C. Merge S&P500&NASDAQ&BTC Datasets**

In [ ]:
result = pd.merge(data, df, how='outer')
result = pd.merge(result, dfn, how='outer')
result

,Date,BTC High,SP500 High,NASDAQ High
0,03/21/2017,1122.430054,"2,381.93",5928.060059
1,03/22/2017,1120.650024,"2,351.81",5825.669922
2,03/23/2017,1058.010010,"2,358.92",5842.819824
3,03/24/2017,1040.469971,"2,356.22",5858.950195
4,03/25/2017,975.760986,NaN,NaN
...,...,...,...,...
1822,03/17/2022,41287.535156,"4,412.67",13620.799805
1823,03/18/2022,42195.746094,"4,465.40",13899.280273
1824,03/19/2022,42316.554688,NaN,NaN
1825,03/20/2022,42241.164062,NaN,NaN


> To fill the missing values of S&P500 and NASDAQ columns, we will dynamically replace them with the average values of previous and next non-missing values so we don't lose much information.

In [ ]:
for i in range (1,1823):
  if pd.isna(result['SP500 High'].iloc[i]):#Check if the value is null
    j=i+1
    while pd.isna(result['SP500 High'].iloc[j]): #search the next non-null value
      j=j+1
    prev=result['SP500 High'].iloc[i-1]
    prev=float(prev.replace(',',''))
    next=result['SP500 High'].iloc[j]
    next=float(next.replace(',',''))
    result['SP500 High'].iloc[i]=str((prev+next)/2) 
for i in range (1,1823):
  if pd.isna(result['NASDAQ High'].iloc[i]):#Check if the value is null
    j=i+1
    while pd.isna(result['NASDAQ High'].iloc[j]): #search the next non-null value
      j=j+1
    prev=result['NASDAQ High'].iloc[i-1]
    next=result['NASDAQ High'].iloc[j]
    result['NASDAQ High'].iloc[i]=float((prev+next)/2) 
data = result.dropna()

for i in range (1824):
  a=data['SP500 High'].iloc[i]
  data['SP500 High'].iloc[i]=float(a.replace(',',''))

data

,Date,BTC High,SP500 High,NASDAQ High
0,03/21/2017,1122.430054,2381.93,5928.060059
1,03/22/2017,1120.650024,2351.81,5825.669922
2,03/23/2017,1058.010010,2358.92,5842.819824
3,03/24/2017,1040.469971,2356.22,5858.950195
4,03/25/2017,975.760986,2350.56,5854.075195
...,...,...,...,...
1819,03/14/2022,39742.500000,4247.57,12918.009766
1820,03/15/2022,39794.628906,4271.05,12973.879883
1821,03/16/2022,41465.453125,4358.9,13440.120117
1822,03/17/2022,41287.535156,4412.67,13620.799805


## **Data Exploration**

### **1. Visualize the Time Series**

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Date'], y=data['BTC High'],
                    mode='lines',
                    name='BTC High',
                    line=dict(color='#a5ade6')))

fig.add_trace(go.Scatter(x=data['Date'], y=data['SP500 High'],
                    mode='lines', name='SP500 High',
                    line=dict(color='#2eb0c7')))

fig.add_trace(go.Scatter(x=data['Date'], y=data['NASDAQ High'],
                    mode='lines', name='NASDAQ High',
                    line=dict(color='#E5EA35')))
fig.update_layout(font_color="#d7dbf5",
                  paper_bgcolor="#020938",  
                  plot_bgcolor="#020938",title="Visualize BTC, NASDAQ and S&P500 evolution")
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        ),
    ),
fig.show()

NameError: ignored

# **Building models**

The data is ready, let’s start the trip of MTS modeling! We will involve the steps below:
*   Causality investigation.
*    Test for stationary.
*   Model Building.






### **Split data into Training and Testing Data**

select 80% of data as the training data and the rest as the test data

In [ ]:
data=data.set_index('Date')

In [ ]:
data["SP500 High"] = pd.to_numeric(data["SP500 High"])
data["NASDAQ High"] = pd.to_numeric(data["NASDAQ High"])

In [ ]:
data_train, data_test = data[0:-90], data[-90:] #The last three months will be used to test the models
data_test

,BTC High,SP500 High,NASDAQ High
Date,,,
12/09/2021,50797.164062,4695.2600,15796.049805
12/10/2021,50015.253906,4713.5700,15677.599609
12/11/2021,49458.210938,4711.9350,15657.329590
12/12/2021,50724.867188,4711.1175,15647.194580
12/13/2021,50205.000000,4710.3000,15637.059570
...,...,...,...
03/14/2022,39742.500000,4247.5700,12918.009766
03/15/2022,39794.628906,4271.0500,12973.879883
03/16/2022,41465.453125,4358.9000,13440.120117


### **Testing Causation using Granger’s Causality Test**

First, we use Granger Causality Test to investigate causality of data. Granger causality is a way to investigate the causality between two variables in a time series which actually means if a particular variable comes before another in the time series. In the MTS, we will test the causality of all combinations of pairs of variables.

The Null Hypothesis of the Granger Causality Test is that lagged column-values do not explain the variation in index, so the column does not cause index. We use grangercausalitytests function in the package statsmodels to do the test and the output of the matrix is the minimum p-value when computes the test for all lags up to maxlag. The critical value we use is 5% and if the p-value of a pair of variables is smaller than 0.05, we could say with 95% confidence that a predictor colomn causes a response index .

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test' #since we are working with multivariate time series, we should use Wald statistic that asymptotically follows a chi-square distribution
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var  for var in variables]
    df.index = [var for var in variables]
    return df
variables=['BTC High','SP500 High','NASDAQ High']
grangers_causation_matrix(data, variables) 

,BTC High,SP500 High,NASDAQ High
BTC High,1.0000,0.0185,0.0002
SP500 High,0.0339,1.0000,0.0016
NASDAQ High,0.1810,0.0012,1.0000


P-Values lesser than the significance level (0.05), implies the Null Hypothesis (the X does not cause Y) can be rejected.


Looking at the P-Values in the table, we can see that S&P500 and NASDAQ series are causing The Bitcoin serie. This makes this system of time series a good candidate for using MTS to predict future values.






### **Check stationarity and stationarize Time Series**

As VAR ,VARIMA and VARIMA with auto_ARIMA requires time series to be stationary, we will use one popular statistical test – Augmented Dickey-Fuller Test (ADF Test) to check the stationary of each variable in the dataset. If the stationarity is not achieved, we need to make the data stationary, such as eliminating the trend and seasonality by differencing and seasonal decomposition.

In the following script, we use adfuller function in the statsmodels package for stationary test of each variables. The Null Hypothesis is that the data has unit root and is not stationary and the significant value is 0.05.

In [ ]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*44)
    print(f' Null Hypothesis: Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Time series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Accepting Null Hypothesis.")
        print(f" => Time series is Non-Stationary.")

In [ ]:
for name, column in data_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "BTC High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -0.4747
 No. Lags Chosen       = 22
 => P-Value = 0.8969. Accepting Null Hypothesis.
 => Time series is Non-Stationary.


    Augmented Dickey-Fuller Test on "SP500 High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 0.2827
 No. Lags Chosen       = 13
 => P-Value = 0.9765. Accepting Null Hypothesis.
 => Time series is Non-Stationary.


    Augmented Dickey-Fuller Test on "NASDAQ High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 0.5645
 No. Lags Chosen       = 13
 => P-Value = 0.9867. Accepting Null Hypothesis.
 => Time series is Non-Stationary.




The ADF test shows that our time series are not stationary. Let’s see what about their first difference.

In [ ]:
data_differenced = data_train.diff().dropna()
for name, column in data_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "BTC High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.3223
 No. Lags Chosen       = 25
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Time series is Stationary.


    Augmented Dickey-Fuller Test on "SP500 High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -9.906
 No. Lags Chosen       = 12
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Time series is Stationary.


    Augmented Dickey-Fuller Test on "NASDAQ High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -10.2559
 No. Lags Chosen       = 12
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Time series is Stationary.




Now, we have stationarized all our time series.

In [ ]:
data_differenced

,BTC High,SP500 High,NASDAQ High
Date,,,
03/22/2017,-1.780029,-30.1200,-102.390137
03/23/2017,-62.640015,7.1100,17.149902
03/24/2017,-17.540039,-2.7000,16.130371
03/25/2017,-64.708984,-5.6600,-4.875000
03/26/2017,32.199036,-2.8300,-2.437500
...,...,...,...
12/04/2021,-3577.488281,2.2850,-94.185059
12/05/2021,-4136.531250,1.1425,-47.092529
12/06/2021,1161.371094,1.1425,-47.092529


# **VAR model**

### **1. Select the Order of VAR model**

In [ ]:
model = VAR(data_differenced)
x = model.select_order(maxlags=12)
x.summary()

,AIC,BIC,FPE,HQIC
0,26.42,26.43,2.967e+11,26.42
1,26.33,26.37*,2.727e+11,26.35*
2,26.33,26.40,2.726e+11,26.36
3,26.33,26.43,2.730e+11,26.37
4,26.32*,26.44,2.682e+11*,26.36
5,26.32,26.47,2.698e+11,26.38
6,26.33,26.51,2.712e+11,26.39
7,26.33,26.54,2.734e+11,26.41
8,26.33,26.57,2.735e+11,26.42
9,26.34,26.60,2.738e+11,26.43


### **2. Training**

In [ ]:
model_fitted = model.fit(4)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 31, Mar, 2022
Time:                     19:19:48
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    26.4257
Nobs:                     1719.00    HQIC:                   26.3478
Log likelihood:          -29885.1    FPE:                2.64758e+11
AIC:                      26.3021    Det(Omega_mle):     2.58841e+11
--------------------------------------------------------------------
Results for equation BTC High
                    coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------
const                 17.303883        19.333039            0.895           0.371
L1.BTC High            0.122386         0.024134            5.071           0.000
L1.SP500 High         -4.567163         2.141952     

### **3. Model Forecasting**

In [ ]:
lag_order = model_fitted.k_ar
lag_order

4

In [ ]:
forecast_input = data_differenced.values[-lag_order:]
forecast_input

array([[-4.13653125e+03,  1.14250000e+00, -4.70925295e+01],
       [ 1.16137109e+03,  1.14250000e+00, -4.70925295e+01],
       [ 1.00526172e+03,  8.14400000e+01,  4.38099610e+02],
       [-7.63406250e+02,  1.10200000e+01,  7.25498040e+01]])

In [ ]:
fc = model_fitted.forecast(y=forecast_input, steps=100)
df_forecast = pd.DataFrame(fc, index=data.index[-100:], columns=data.columns + '_1d')
df_forecast

,BTC High_1d,SP500 High_1d,NASDAQ High_1d
Date,,,
12/09/2021,-813.226814,3.066020,27.774097
12/10/2021,101.456193,5.915249,25.025596
12/11/2021,267.741782,0.957114,-1.928001
12/12/2021,6.563470,1.587888,6.707615
12/13/2021,-64.756206,0.582468,3.227580
...,...,...,...
03/14/2022,28.893350,1.372957,5.798973
03/15/2022,28.893350,1.372957,5.798973
03/16/2022,28.893350,1.372957,5.798973


In [ ]:
def invert_transformation(df_train, df_forecast):
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:
      df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [ ]:
df_results = invert_transformation(data_train, df_forecast)
df_results

,BTC High_1d,SP500 High_1d,NASDAQ High_1d,BTC High_forecast,SP500 High_forecast,NASDAQ High_forecast
Date,,,,,,
12/09/2021,-813.226814,3.066020,27.774097,50358.148186,4708.126020,15820.413745
12/10/2021,101.456193,5.915249,25.025596,50459.604379,4714.041269,15845.439342
12/11/2021,267.741782,0.957114,-1.928001,50727.346161,4714.998384,15843.511340
12/12/2021,6.563470,1.587888,6.707615,50733.909631,4716.586272,15850.218956
12/13/2021,-64.756206,0.582468,3.227580,50669.153424,4717.168739,15853.446535
...,...,...,...,...,...,...
03/14/2022,28.893350,1.372957,5.798973,53298.276004,4841.688505,16379.366989
03/15/2022,28.893350,1.372957,5.798973,53327.169354,4843.061462,16385.165961
03/16/2022,28.893350,1.372957,5.798973,53356.062704,4844.434419,16390.964934


### **4. Visualize forcast VS actual**








In [ ]:
fig = go.Figure()
fig1 = go.Figure()
fig2 = go.Figure()
fig.add_trace(go.Scatter(x=data_test.index, y=data_test['BTC High'],
                    mode='lines',
                    name='BTC High',
                    line=dict(color='#a03fba')))

fig1.add_trace(go.Scatter(x=data_test.index, y=data_test['NASDAQ High'],
                    mode='lines',
                    name='NASDAQ High',
                    line=dict(color='#a03fba')))
fig2.add_trace(go.Scatter(x=data_test.index, y=data_test['SP500 High'],
                    mode='lines', name='SP500 High',
                    line=dict(color='#a03fba')))
fig.add_trace(go.Scatter(x=data_test.index, y=df_results['BTC High_forecast'],
                    mode='lines',
                    name='BTC High forecast',
                    line=dict(color='#2eb0c7')))

fig1.add_trace(go.Scatter(x=data_test.index, y=df_results['NASDAQ High_forecast'],
                    mode='lines',
                    name='NASDAQ High',
                    line=dict(color='#2eb0c7')))
fig2.add_trace(go.Scatter(x=data_test.index, y=df_results['SP500 High_forecast'],
                    mode='lines', name='SP500 High forecast',
                    line=dict(color='#2eb0c7')))
for plot in [fig,fig1,fig2]:
  plot.update_layout(font_color="#d7dbf5",
                  paper_bgcolor="#020938",  
                  plot_bgcolor="#020938",title="Forcast VS Actual")
  plot.update_layout(
      xaxis=dict(
          showline=True,
          showgrid=False,
          showticklabels=True,
          ),
      )

### **5. Forecast Evaluation For VAR model**

In [ ]:
from statsmodels.tsa.stattools import acf
def adjust(val, length= 6): return str(val).ljust(length)
def forecast_accuracy(forecast, actual):
    m=np.abs(forecast - actual)/np.abs(actual)
    mape = np.mean(m)  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE

    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse})

print('Forecast Accuracy of: BTC HIGH')
accuracy_prod = forecast_accuracy(df_results['BTC High_forecast'].values, data_test['BTC High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))


print('\nForecast Accuracy of: SP500')
accuracy_prod = forecast_accuracy(df_results['SP500 High_forecast'].values, data_test['SP500 High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: NASDAQ')
accuracy_prod = forecast_accuracy(df_results['NASDAQ High_forecast'].values, data_test['SP500 High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

Forecast Accuracy of: BTC HIGH
mape   :  0.2073
me     :  8450.0093
mae    :  8506.7316
mpe    :  0.2062
rmse   :  9723.6939

Forecast Accuracy of: SP500
mape   :  0.0562
me     :  234.0657
mae    :  248.6144
mpe    :  0.0532
rmse   :  310.025

Forecast Accuracy of: NASDAQ
mape   :  2.5517
me     :  11570.4642
mae    :  11570.4642
mpe    :  2.5517
rmse   :  11575.0236


# **VARMA model**


### **1. Select the Order of** **VARMA model**




In [ ]:
param_grid = {'p': [1,2,3], 'q':[1,2,3], 'tr': ['n','c','t','ct']}
pg = list(ParameterGrid(param_grid))

In [ ]:
def inverse_diff(actual_df, pred_df):
    '''
    Transforms the differentiated values back
    
    Args:
        actual dataframe (float64): Values of the columns, numpy array of floats 
        predicted dataframe (float64): Values of the columns, numpy array of floats 
    
    Returns:
        Dataframe with the predicted values
    '''
    df_res = pred_df.copy()
    columns = actual_df.columns
    for col in columns: 
        df_res[str(col)+'_1st_inv_diff'] = actual_df[col].iloc[-1] + df_res[str(col)].cumsum()
    return df_res

In [ ]:
df_results_VARMA = pd.DataFrame(columns=['p', 'q', 'tr','RMSE btchigh','RMSE sphigh','RMSE nashigh'])

for a,b in enumerate(pg):
    p = b.get('p')
    q = b.get('q')
    tr = b.get('tr')
    model = VARMAX(data_differenced, order=(p,q), trend=tr)
    model_fit=model.fit()
    z = model_fit.forecast(y=data_differenced[['BTC High', 'SP500 High','NASDAQ High']],steps=len(data_test))
    df_pred = pd.DataFrame(z, columns=['BTC High', 'SP500 High','NASDAQ High'])
    res = inverse_diff(data[['BTC High', 'SP500 High','NASDAQ High']],df_pred)
    highrmse = np.sqrt(metrics.mean_squared_error(data_test['BTC High'], res['BTC High_1st_inv_diff']))
    sphighrmse = np.sqrt(metrics.mean_squared_error(data_test['SP500 High'], res['SP500 High_1st_inv_diff']))
    nashighrmse = np.sqrt(metrics.mean_squared_error(data_test['NASDAQ High'], res['NASDAQ High_1st_inv_diff']))
    df_results_VARMA = df_results_VARMA.append({'p': p, 'q': q, 'tr': tr,'RMSE btchigh':highrmse,'RMSE sphigh':sphighrmse,'RMSE nashigh':nashighrmse}, ignore_index=True)

In [ ]:
df_results_VARMA.sort_values(by=['RMSE btchigh','RMSE sphigh','RMSE nashigh']).head()


,p,q,tr,RMSE btchigh,RMSE sphigh,RMSE nashigh
0,1,1,n,4393.935640,179.334239,1035.843588
28,3,2,n,4451.028184,180.104921,1036.044320
32,3,3,n,4451.891150,180.947669,1040.608814
24,3,1,n,4454.463132,180.189992,1037.136728
12,2,1,n,4464.792699,180.714492,1036.836691


In [ ]:
best_values_VARMA = df_results_VARMA.sort_values(by=['RMSE btchigh','RMSE sphigh']).head(1)
best_values_VARMA

,p,q,tr,RMSE btchigh,RMSE sphigh,RMSE nashigh
0,1,1,n,4393.93564,179.334239,1035.843588


Finaly , we got the best order of VARMA model 

In [ ]:
p_value_VARMA = best_values_VARMA['p'].iloc[0]
q_value_VARMA = best_values_VARMA['q'].iloc[0] 
tr_value_VARMA = best_values_VARMA['tr'].iloc[0] 

print("p_value_VARMA: ", p_value_VARMA)
print("q_value_VARMA: ", q_value_VARMA)
print("tr_value_VARMA: ", tr_value_VARMA)

p_value_VARMA:  1
q_value_VARMA:  1
tr_value_VARMA:  n


### **2. Forcasting of VARMA model** 

In this section, we will use forcast() function of VARMA to get the forecast 
results and then evaluate the forecasts with data_test.

In [ ]:
model_fit = VARMAX(data_differenced[['BTC High', 'SP500 High','NASDAQ High']], 
               order=(p_value_VARMA, q_value_VARMA),trends = tr_value_VARMA).fit(disp=False)
result = model_fit.forecast(steps = len(data_test))

In [ ]:
res = inverse_diff(data[['BTC High', 'SP500 High','NASDAQ High']], result)
res.head()

,BTC High,SP500 High,NASDAQ High,BTC High_1st_inv_diff,SP500 High_1st_inv_diff,NASDAQ High_1st_inv_diff
2021-12-09,-23.886660,8.746860,34.406739,42171.859434,4474.146860,13933.687012
2021-12-10,50.079528,4.588038,14.634786,42221.938962,4478.734898,13948.321798
2021-12-11,34.420078,3.154413,9.786654,42256.359040,4481.889311,13958.108451
2021-12-12,29.793986,2.518867,8.136700,42286.153026,4484.408178,13966.245151
2021-12-13,28.551595,2.252030,7.565560,42314.704621,4486.660209,13973.810711


In [ ]:
res["new_index"] = range(len(data_train), len(data))
res =res.set_index("new_index")
res

,BTC High,SP500 High,NASDAQ High,BTC High_1st_inv_diff,SP500 High_1st_inv_diff,NASDAQ High_1st_inv_diff
new_index,,,,,,
1724,-23.886660,8.746860,34.406739,42171.859434,4474.146860,13933.687012
1725,50.079528,4.588038,14.634786,42221.938962,4478.734898,13948.321798
1726,34.420078,3.154413,9.786654,42256.359040,4481.889311,13958.108451
1727,29.793986,2.518867,8.136700,42286.153026,4484.408178,13966.245151
1728,28.551595,2.252030,7.565560,42314.704621,4486.660209,13973.810711
...,...,...,...,...,...,...
1819,28.142152,2.075126,7.258417,44875.744671,4675.608368,14634.493964
1820,28.142152,2.075126,7.258417,44903.886823,4677.683494,14641.752380
1821,28.142152,2.075126,7.258417,44932.028975,4679.758619,14649.010797


### **3. Visualize forcast VS actual**

Visualize the forecast with actual values:


In [ ]:
fig = go.Figure()
fig3 = go.Figure()
fig2 = go.Figure()

fig.add_trace(go.Scatter(x=data_test.index, y=data_test['BTC High'],
                    mode='lines',
                    name='BTC High',
                    line=dict(color='#a03fba')))

fig2.add_trace(go.Scatter(x=data_test.index, y=data_test['SP500 High'],
                    mode='lines', name='SP500 High',
                    line=dict(color='#a03fba')))
fig3.add_trace(go.Scatter(x=data_test.index, y=data_test['NASDAQ High'],
                    mode='lines', name='NASDAQ High',
                    line=dict(color='#a03fba')))
fig.add_trace(go.Scatter(x=data_test.index, y=res['BTC High_1st_inv_diff'],
                    mode='lines',
                    name='BTC High forecast',
                    line=dict(color='#2eb0c7')))


fig2.add_trace(go.Scatter(x=data_test.index, y=res['SP500 High_1st_inv_diff'],
                    mode='lines', name='SP500 High forecast',
                    line=dict(color='#2eb0c7')))

fig3.add_trace(go.Scatter(x=data_test.index, y=res['NASDAQ High_1st_inv_diff'],
                    mode='lines', name='NASDAQ High forecast',
                    line=dict(color='#2eb0c7')))
for plot in [fig,fig2,fig3]:
  plot.update_layout(font_color="#d7dbf5",
                  paper_bgcolor="#020938",  
                  plot_bgcolor="#020938",title="Forcast VS Actual")
  plot.update_layout(
      xaxis=dict(
          showline=True,
          showgrid=False,
          showticklabels=True,
          ),
      )
  plot.show()

### **4. Forecast Evaluation**

Then, use forcast_accuracy() function of hana-ml to evaluate the forecasts with metric ‘rmse’,'MAPE','MAE','ME','MPE'.


In [ ]:
from statsmodels.tsa.stattools import acf
def adjust(val, length= 6): return str(val).ljust(length)
def forecast_accuracy(forecast, actual):
    m=np.abs(forecast - actual)/np.abs(actual)
    mape = np.mean(m)  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE

    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse})

print('Forecast Accuracy of: BTC HIGH')
accuracy_prod = forecast_accuracy(res['BTC High_1st_inv_diff'].values, data_test['BTC High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))


print('\nForecast Accuracy of: SP500')
accuracy_prod = forecast_accuracy(res['SP500 High_1st_inv_diff'].values, data_test['SP500 High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))
    
print('\nForecast Accuracy of: NASDAQ')
accuracy_prod = forecast_accuracy(res['NASDAQ High_1st_inv_diff'].values, data_test['NASDAQ High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

Forecast Accuracy of: BTC HIGH
mape   :  0.0913
me     :  62.2846
mae    :  3983.0936
mpe    :  0.0117
rmse   :  4793.1806

Forecast Accuracy of: SP500
mape   :  0.0445
me     :  35.9548
mae    :  200.7016
mpe    :  0.0097
rmse   :  225.1357

Forecast Accuracy of: NASDAQ
mape   :  0.0643
me     :  -184.5716
mae    :  941.0266
mpe    :  -0.0083
rmse   :  1097.0827


# **VARMA with AUTO_ARIMA MODEL**

### **1. Select the order of VARMA with AUTO_ARIMA model**

In [ ]:
pq = []
for name, column in data_differenced[['BTC High', 'SP500 High', 'NASDAQ High']].iteritems():
    print(f'Searching order of p and q for : {name}')
    stepwise_model = auto_arima(data_differenced[name],start_p=1, start_q=1,max_p=7, max_q=7, seasonal=False,
        trace=True,error_action='ignore',suppress_warnings=True, stepwise=True,maxiter=1000)
    parameter = stepwise_model.get_params().get('order')
    print(f'optimal order for:{name} is: {parameter} \n\n')
    pq.append(stepwise_model.get_params().get('order'))

Searching order of p and q for : BTC High
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=27935.197, Time=0.21 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=27957.548, Time=0.06 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=27933.247, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=27934.290, Time=0.16 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=27935.162, Time=0.24 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=27924.460, Time=1.18 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=27922.898, Time=1.15 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=27934.438, Time=0.24 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=27914.288, Time=1.11 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=27913.050, Time=0.28 sec
 ARIMA(5,0,0)(0,0,0)[0]             : AIC=27914.255, Time=0.36 sec
 ARIMA(5,0,1)(0,0,0)[0]             : AIC=27916.248, Time=1.23 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : AIC=27913.609, Time=0.58 sec

Best model:  ARIMA(4,0,0)(0,0,0)[0]        

In [ ]:
pq

[(4, 0, 0), (2, 0, 0), (2, 0, 0)]

In [ ]:
df_results_VARMA_2 = pd.DataFrame(columns=['p', 'q','BTC High2','SP500 High2','NASDAQ High2'])


In [ ]:
for i in pq:
    if i[0]== 0 and i[2] ==0:
        pass
    else:
        print(f' Running for {i}')
        model = VARMAX(data_differenced[['BTC High', 'SP500 High','NASDAQ High']], order=(i[0],i[2]))
        model_fit=model.fit(disp=False)
        result = model_fit.forecast(steps = len(data_test))
        inv_res = inverse_diff(data[['BTC High', 'SP500 High','NASDAQ High']], result)
        btcrmse = np.sqrt(metrics.mean_squared_error(data_test['BTC High'], inv_res['BTC High_1st_inv_diff']))
        sprmse = np.sqrt(metrics.mean_squared_error(data_test['SP500 High'], inv_res['SP500 High_1st_inv_diff']))
        NASrmse = np.sqrt(metrics.mean_squared_error(data_test['NASDAQ High'], inv_res['NASDAQ High_1st_inv_diff']))
        df_results_VARMA_2 = df_results_VARMA_2.append({'p': i[0], 'q': i[2], 'BTC High2':btcrmse,
                                                        'SP500 High2':sprmse,
                                                        'NASDAQ High2':NASrmse }, ignore_index=True)

 Running for (4, 0, 0)
 Running for (2, 0, 0)
 Running for (2, 0, 0)


In [ ]:
df_results_VARMA_2.sort_values(by=['BTC High2', 'SP500 High2', 'NASDAQ High2'])


,p,q,BTC High2,SP500 High2,NASDAQ High2
1,2.0,0.0,4812.797119,202.656662,1070.559537
2,2.0,0.0,4812.797119,202.656662,1070.559537
0,4.0,0.0,4841.961174,202.693947,1073.626637


In [ ]:
best_values_VAR_2 = df_results_VARMA_2.sort_values(by=['BTC High2', 'SP500 High2', 'NASDAQ High2']).head(1)
best_values_VAR_2

,p,q,BTC High2,SP500 High2,NASDAQ High2
1,2.0,0.0,4812.797119,202.656662,1070.559537


the best order is (2,0,0)

In [ ]:
p_value_VARMA_2 = best_values_VAR_2['p'].iloc[0]
q_value_VARMA_2 = best_values_VAR_2['q'].iloc[0] 

print("p_value_VARMA_2: ", p_value_VARMA_2)
print("q_value_VARMA_2: ", q_value_VARMA_2)

p_value_VARMA_2:  2.0
q_value_VARMA_2:  0.0


### **2. Model Forecasting**

In [ ]:
model = VARMAX(data_differenced[['BTC High', 'SP500 High', 'NASDAQ High']], 
               order=(int(p_value_VARMA_2),int(q_value_VARMA_2))).fit(disp=False)
result = model.forecast(steps = len(data_test))

In [ ]:
result2 = inverse_diff(data[['BTC High', 'SP500 High', 'NASDAQ High']],result)
result2.head()

,BTC High,SP500 High,NASDAQ High,BTC High_1st_inv_diff,SP500 High_1st_inv_diff,NASDAQ High_1st_inv_diff
2021-12-09,-165.872119,8.902559,37.410844,42029.873975,4474.302559,13936.691117
2021-12-10,-15.368158,3.250582,14.284821,42014.505817,4477.553141,13950.975939
2021-12-11,24.108793,2.160438,8.663762,42038.614610,4479.713579,13959.639701
2021-12-12,27.890976,1.614295,6.635483,42066.505587,4481.327874,13966.275183
2021-12-13,29.423577,1.446608,6.022081,42095.929164,4482.774482,13972.297265


### **3. Visualize the forecast with actual values of VARMA with auto_ARIMA model:**

In [ ]:
fig = go.Figure()

fig2 = go.Figure()

fig.add_trace(go.Scatter(x=data_test.index, y=data_test['BTC High'],
                    mode='lines',
                    name='BTC High',
                    line=dict(color='#a03fba')))

fig2.add_trace(go.Scatter(x=data_test.index, y=data_test['SP500 High'],
                    mode='lines', name='SP500 High',
                    line=dict(color='#a03fba')))
fig.add_trace(go.Scatter(x=data_test.index, y=result2['BTC High_1st_inv_diff'],
                    mode='lines',
                    name='BTC High forecast',
                    line=dict(color='#2eb0c7')))


fig2.add_trace(go.Scatter(x=data_test.index, y=result2['SP500 High_1st_inv_diff'],
                    mode='lines', name='SP500 High forecast',
                    line=dict(color='#2eb0c7')))
for plot in [fig,fig2]:
  plot.update_layout(font_color="#d7dbf5",
                  paper_bgcolor="#020938",  
                  plot_bgcolor="#020938",title="Forcast VS Actual")
  plot.update_layout(
      xaxis=dict(
          showline=True,
          showgrid=False,
          showticklabels=True,
          ),
      )
  plot.show()

### **4. Forcast EVALUATION**

In [ ]:
from statsmodels.tsa.stattools import acf
def adjust(val, length= 6): return str(val).ljust(length)
def forecast_accuracy(forecast, actual):
    m=np.abs(forecast - actual)/np.abs(actual)
    mape = np.mean(m)  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE

    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse})

print('Forecast Accuracy of: BTC HIGH')
accuracy_prod = forecast_accuracy(result2['BTC High_1st_inv_diff'].values, data_test['BTC High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))


print('\nForecast Accuracy of: SP500')
accuracy_prod = forecast_accuracy(result2['SP500 High_1st_inv_diff'].values, data_test['SP500 High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: NASDAQ')
accuracy_prod = forecast_accuracy(result2['NASDAQ High_1st_inv_diff'].values, data_test['NASDAQ High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

Forecast Accuracy of: BTC HIGH
mape   :  0.091
me     :  -114.3411
mae    :  3983.2708
mpe    :  0.0077
rmse   :  4812.7971

Forecast Accuracy of: SP500
mape   :  0.0396
me     :  -1.1233
mae    :  179.5667
mpe    :  0.0014
rmse   :  202.6567

Forecast Accuracy of: NASDAQ
mape   :  0.0612
me     :  -255.9004
mae    :  900.287
mpe    :  -0.0134
rmse   :  1070.5595


# **Conclusion**

After studying the causality beteween the three time series, we found out that both S&P500 and NASDAQ series are causing the evolution of Bitcoin.
Then we built VAR,VARMA and VARMA with AUTO-ARIMA and compared performances.
We conclude that the best model is VARMA with auto-ARIMA since it has the least bitcoin price forcasting error.